In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

Visualize the data generated from AI

In [3]:
df = read_csv('dataset.csv')
df.head()

NameError: name 'read_csv' is not defined